In [1]:
import ROOT
import logging
from array import array
import sys, re
import os
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

Welcome to JupyROOT 6.28/00


In [2]:
def getHistograms(variableName, variableStyle, dataset, j, min, max, sample="signal"):

    h = ROOT.TH1F(
        sample + "_" + variableName,
        sample + "_" + variableName,
        100,
        min,
        max,
    )

    for i in range(len(dataset)):
        h.Fill(dataset[i][j])
    h.GetXaxis().SetTitle(variableStyle)

    return h

In [3]:
def plotVars(dictionary, signal_dataset, bkg_dataset):

    variableName = [*dictionary.keys()]
    variableStyle = [*dictionary.values()]

    for j in range(len(variableName)):
        max = 1.0
        min =-1.0
        for i in range(len(signal_dataset)):
           max=np.maximum(signal_dataset[i][j],max)
           min=np.minimum(signal_dataset[i][j],min)

        logging.info("range of " + variableName[j] + " : [" + str(min) + ", " + str(max) + "]")
        logging.info("plotting " + variableName[j])

        hist_signal = getHistograms(
            variableName[j], variableStyle[j], signal_dataset, j, min, max, sample="signal"
        )
        hist_bkg = getHistograms(
            variableName[j], variableStyle[j], bkg_dataset, j, min, max, sample="bkg"
        )
        canvas = ROOT.TCanvas("canvas", "canvas", 600, 600)
        canvas.SetTickx()
        canvas.SetTicky()
        ROOT.gPad.SetLeftMargin(0.14)
        ROOT.gStyle.SetOptStat(0)
        canvas.SetName(variableName[j])
        if hist_bkg.Integral() != 0 and hist_bkg.Integral() != 0:
            hist_signal.Scale(1 / float(hist_signal.Integral()))
            hist_bkg.Scale(1 / float(hist_bkg.Integral()))
        hist_signal.SetLineColor(2)
        hist_bkg.SetLineColor(4)
        hist_signal.SetMarkerColor(2)
        hist_signal.GetYaxis().SetTitle("Arbitrary units")
        hist_signal.GetYaxis().SetTitleOffset(1.7)
        hist_signal.GetXaxis().SetTitleOffset(1.2)
        hist_bkg.SetMarkerColor(4)
        hist_signal.SetFillColor(2)
        hist_bkg.SetFillColor(4)
        hist_signal.SetFillStyle(3004)
        hist_signal.SetTitle("")
        hist_bkg.SetFillStyle(3005)
        hist_bkg.SetTitle("")
        ymax = np.maximum(hist_signal.GetMaximum(), hist_bkg.GetMaximum())
        hist_signal.GetYaxis().SetRangeUser(0, 1.5 * ymax)
        #hist_signal.GetXaxis().SetRangeUser(-1, 1)

        hist_signal.Draw("hist")
        hist_bkg.Draw("hist same")
        legend = ROOT.TLegend(0.60, 0.74, 0.85, 0.86)
        legend.SetBorderSize(0)
        legend.SetTextAlign(12)
        legend.SetTextFont(42)
        legend.SetTextSize(0.035)
        legend.SetLineColor(0)
        legend.SetLineStyle(0)
        legend.SetLineWidth(0)
        legend.SetFillColor(0)
        legend.SetFillStyle(1001)

        legend.AddEntry(hist_signal, "Signal", "F")
        legend.AddEntry(hist_bkg, "Background", "F")
        legend.Draw()
        canvas.Draw()

        canvas.Print("./images/variables/" + variableName[j] +".pdf")